In [1]:
import tweepy
from os import environ,listdir
import json, pandas as pd
from os.path import expanduser, join
from datetime import datetime as dt
from datetime import timedelta as td

In [2]:
token = environ.get('BEARER_TOKEN')

# cli = tweepy.Client(bearer_token = token,wait_on_rate_limit=True)

In [3]:
auth = tweepy.OAuthHandler(environ.get("API_KEY"), environ.get("API_KEY_SECRET"))
auth.set_access_token(environ.get("ACCESS_TOKEN"),environ.get("ACCESS_TOKEN_SECRET"))

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
twitter_fields = 'attachments,context_annotations,conversation_id,created_at,entities,in_reply_to_user_id,lang,public_metrics,referenced_tweets,source'
user_fields = 'description,created_at,location,entities,public_metrics,verified'
place_fields = 'country,name'

In [12]:
query = '"UNESCO" -from:UNESCO -from:UN -from:UN_Women -from:UNICEF -from:UNIDO (lang:en OR lang:fr) -has:videos -retweets_of:UNESCO'

In [13]:
len(query)

123

In [21]:
start = dt.now() - td(days = 365*2)
end = dt.now() - td(days = 365)

In [10]:
data = []

In [22]:
d = api.search_full_archive('dashboard',query, toDate = end.strftime('%Y%m%d%H%m'), fromDate = start.strftime('%Y%m%d%H%m'))

In [23]:
data.append(d)

In [28]:
df = pd.DataFrame([d._json for d in data[0]])

In [29]:
df = pd.concat((df, pd.DataFrame([d._json for d in data[1]])))

In [30]:
df = pd.concat((df, pd.DataFrame([d._json for d in data[2]])))

In [31]:
full_text = pd.json_normalize(df['extended_tweet'])

In [32]:
full_text.dropna(subset='full_text', inplace = True)#[full_text['full_text'].fillna('').str.contains('UNESCO',regex=False)]

In [34]:
df.reset_index(inplace=True)

In [35]:
df = pd.concat((df,pd.json_normalize(df['extended_tweet'])),axis=1)

In [36]:
df['text'].where(~df['truncated'],df['full_text'], inplace = True)

In [38]:
df.keys()

Index(['index', 'created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'extended_tweet', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'possibly_sensitive', 'edit_history',
       'edit_controls', 'editable', 'filter_level', 'lang', 'matching_rules',
       'retweeted_status', 'display_text_range', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink',
       'extended_entities', 'full_text', 'display_text_range',
       'entities.hashtags', 'entities.urls', 'entities.user_mentions',
       'entities.symbols', 'entities.media', 'extended_entities.media'],
      dtype='object')

In [47]:
df.text.to_csv('eg.csv',index = False)#.find('UNESCO')

In [125]:
links = r'https?://[\w\.\d/]+'
hashtags = r'#[\w\d]+'
usernames = r'@[\w\d_]+'
df3['text'] = df3['text'].str.replace(links,'', regex = True).str.replace(hashtags,'', regex = True).str.replace(usernames, '',regex = True).str.replace(r'\s+',' ', regex = True)

In [128]:
from textblob import TextBlob
 

In [127]:
sentiment = [TextBlob(i).sentiment for i in list(df3[df3['lang']=='en'].text)]

['RT : "Citizen engagement - a pillar to achieving " says co-organising with …',
 'RT : Africa should approach holistically to ensure that human resources and the gap between actual capacities on th…',
 " Read the tweet again... I am referring to UNESCO's single minded approach... I'm potentially agreeing with you. The value of a place is the experience in the street not the view from an office in Florence.",
 'Dr. Saba Al Heialy was recently honoured as ‘International Rising Talent’ for her research into the link between obesity and asthma at the 21st L’Oréal-UNESCO for Women in Science Award Ceremony. ',
 '“Working with young people not just for young people” says \u2066\u2069 who thanks at Global Committee ',
 'RT : Together, we…',
 "RT : On 's World Day for Peace and Development, we reflect on 40+ years of research at into…",
 'RT : We know what WON’T be on their agenda - making the Azerbaijani government accountable for the secret erasure of 28,000…',
 ' Congratulations Caroline. 